In [3]:
%%!
pip install --trusted-host pypi.org ipython-sql

['Requirement already satisfied: ipython-sql in c:\\users\\beyza\\anaconda3\\lib\\site-packages (0.5.0)',
 'Requirement already satisfied: prettytable in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from ipython-sql) (3.6.0)',
 'Requirement already satisfied: ipython in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from ipython-sql) (8.25.0)',
 'Requirement already satisfied: sqlalchemy>=2.0 in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from ipython-sql) (2.0.30)',
 'Requirement already satisfied: sqlparse in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from ipython-sql) (0.5.3)',
 'Requirement already satisfied: six in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from ipython-sql) (1.16.0)',
 'Requirement already satisfied: ipython-genutils in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from ipython-sql) (0.2.0)',
 'Requirement already satisfied: typing-extensions>=4.6.0 in c:\\users\\beyza\\anaconda3\\lib\\site-packages (from sqlalchemy>=2.0->ipython-sql) (4

## Soru 1 — Tüm verileri göster

In [303]:
import sqlite3
import pandas as pd

In [305]:
conn = sqlite3.connect("sqlite-sakila.db")

In [307]:
film_df = pd.read_sql("SELECT * FROM film", conn)
print(film_df)

     film_id              title  \
0          1   ACADEMY DINOSAUR   
1          2     ACE GOLDFINGER   
2          3   ADAPTATION HOLES   
3          4   AFFAIR PREJUDICE   
4          5        AFRICAN EGG   
..       ...                ...   
995      996     YOUNG LANGUAGE   
996      997         YOUTH KICK   
997      998       ZHIVAGO CORE   
998      999  ZOOLANDER FICTION   
999     1000          ZORRO ARK   

                                           description release_year  \
0    A Epic Drama of a Feminist And a Mad Scientist...         2006   
1    A Astounding Epistle of a Database Administrat...         2006   
2    A Astounding Reflection of a Lumberjack And a ...         2006   
3    A Fanciful Documentary of a Frisbee And a Lumb...         2006   
4    A Fast-Paced Documentary of a Pastry Chef And ...         2006   
..                                                 ...          ...   
995  A Unbelieveable Yarn of a Boat And a Database ...         2006   
996  A Touc

## Soru 2 — Her filmdeki oyuncuları listele

In [310]:
actor_df = pd.read_sql("SELECT * FROM actor", conn)
film_actor_df = pd.read_sql("SELECT * FROM film_actor", conn)

film_actor_list = film_df.merge(film_actor_df, on="film_id") \
                         .merge(actor_df, on="actor_id") \
                         [["title", "first_name", "last_name"]]
film_actor_list

,title,first_name,last_name
0,ACADEMY DINOSAUR,PENELOPE,GUINESS
1,ACADEMY DINOSAUR,CHRISTIAN,GABLE
2,ACADEMY DINOSAUR,LUCILLE,TRACY
3,ACADEMY DINOSAUR,SANDRA,PECK
4,ACADEMY DINOSAUR,JOHNNY,CAGE
...,...,...,...
5457,ZOOLANDER FICTION,WHOOPI,HURT
5458,ZOOLANDER FICTION,JADA,RYDER
5459,ZORRO ARK,IAN,TANDY
5460,ZORRO ARK,NICK,DEGENERES


## Soru 3 — Her filmde kaç oyuncu oynadı?

In [313]:
film_actor_count = film_actor_df.groupby("film_id")["actor_id"].count().reset_index()
film_actor_count = film_actor_count.merge(film_df[["film_id", "title"]], on="film_id")
film_actor_count.rename(columns={"actor_id": "actor_count"}, inplace=True)
film_actor_count


,film_id,actor_count,title
0,1,10,ACADEMY DINOSAUR
1,2,4,ACE GOLDFINGER
2,3,5,ADAPTATION HOLES
3,4,5,AFFAIR PREJUDICE
4,5,5,AFRICAN EGG
...,...,...,...
992,996,5,YOUNG LANGUAGE
993,997,5,YOUTH KICK
994,998,6,ZHIVAGO CORE
995,999,5,ZOOLANDER FICTION


## Soru 4 — Her oyuncu kaç filmde oynadı?

In [316]:
actor_film_count = film_actor_df.groupby("actor_id")["film_id"].count().reset_index()
actor_film_count = actor_film_count.merge(actor_df, on="actor_id")
actor_film_count.rename(columns={"film_id": "film_count"}, inplace=True)
actor_film_count[["first_name", "last_name", "film_count"]]


,first_name,last_name,film_count
0,PENELOPE,GUINESS,19
1,NICK,WAHLBERG,25
2,ED,CHASE,22
3,JENNIFER,DAVIS,22
4,JOHNNY,LOLLOBRIGIDA,29
...,...,...,...
195,BELA,WALKEN,30
196,REESE,WEST,33
197,MARY,KEITEL,40
198,JULIA,FAWCETT,15


## Soru 5: Envanterde olmayan filmler var mı ve varsa kaç tane?

In [319]:
inventory_df = pd.read_sql("SELECT * FROM inventory", conn)

films_without_inventory = film_df[~film_df["film_id"].isin(inventory_df["film_id"])]
films_without_inventory_count = films_without_inventory.shape[0]
films_without_inventory_count


42

### Soru 6: Kiralanabilir olan her filmin kaç kez kiralandığını ve toplam gelirlerini getirin

In [458]:
film_rentals = film_df.merge(inventory_df, on="film_id") \
                      .merge(rental_df, on="inventory_id")

payment_clean = payment_df[["rental_id", "amount"]]
film_rentals = film_rentals.merge(payment_clean, on="rental_id")

film_stats = film_rentals.groupby("title").agg(
    rental_count=("rental_id", "count"),
    total_revenue=("amount", "sum")
).reset_index()

film_stats.head()


,title,rental_count,total_revenue
0,ACADEMY DINOSAUR,23,36.77
1,ACE GOLDFINGER,7,52.93
2,ADAPTATION HOLES,12,37.88
3,AFFAIR PREJUDICE,23,91.77
4,AFRICAN EGG,12,51.88


### Soru 7: Envanterde olmayan filmlerin kira oranlarını getirin

In [461]:
films_no_inventory = film_df[~film_df["film_id"].isin(inventory_df["film_id"])]
films_no_inventory[["title", "rental_rate"]]


,title,rental_rate
13,ALICE FANTASIA,0.99
32,APOLLO TEEN,2.99
35,ARGONAUTS TOWN,0.99
37,ARK RIDGEMONT,0.99
40,ARSENIC INDEPENDENCE,0.99
86,BOONDOCK BALLROOM,0.99
107,BUTCH PANTHER,0.99
127,CATCH AMISTAD,0.99
143,CHINATOWN GLADIATOR,4.99
147,CHOCOLATE DUCK,2.99


### Soru 8: Birden fazla DVD'yi iade etmeyen kaç müşteri var?

In [463]:
customers_not_returned = rental_df[rental_df["return_date"].isnull()]
customers_not_returned_count = customers_not_returned.groupby("customer_id")["rental_id"].count().reset_index()
customers_not_returned_count = customers_not_returned_count[customers_not_returned_count["rental_id"] > 1]
customers_not_returned_count


,customer_id,rental_id
4,15,2
12,42,2
13,43,2
16,53,2
19,60,2
23,75,3
32,107,2
41,155,2
43,163,2
45,175,2


### Soru 9: Her müşteri kaç film kiraladı?

In [466]:
customer_df = pd.read_sql("SELECT * FROM customer", conn)

customer_rentals = rental_df.groupby("customer_id")["rental_id"].count().reset_index()
customer_rentals = customer_rentals.merge(customer_df, on="customer_id")
customer_rentals.rename(columns={"rental_id": "rental_count"}, inplace=True)
customer_rentals[["first_name", "last_name", "rental_count"]]


,first_name,last_name,rental_count
0,MARY,SMITH,32
1,PATRICIA,JOHNSON,27
2,LINDA,WILLIAMS,26
3,BARBARA,JONES,22
4,ELIZABETH,BROWN,38
...,...,...,...
594,TERRENCE,GUNDERSON,30
595,ENRIQUE,FORSYTHE,28
596,FREDDIE,DUGGAN,25
597,WADE,DELVALLE,22


### Soru 10: Türlerine göre en çok kiralanan filmler ve bunlara ne kadar ödendi?

In [469]:
film_cat = film_category.merge(category, on="category_id") \
                           .merge(film, on="film_id")

film_rentals = rental.merge(inventory, on="inventory_id") \
                        .merge(payment, on="rental_id") \
                        .merge(film_cat, on="film_id")

result = film_rentals.groupby(["name", "title"]).agg(
    rental_count=("rental_id", "count"),
    total_revenue=("amount", "sum")
).reset_index()

result.sort_values(["name", "rental_count"], ascending=[True, False]).head()

,name,title,rental_count,total_revenue
46,Action,RUGRATS SHAKESPEARE,30,70.70
53,Action,SUSPECTS QUILLS,30,133.70
29,Action,HANDICAP BOONDOCK,28,63.72
52,Action,STORY SIDE,28,39.72
54,Action,TRIP NEWTON,28,145.72


## Soru 11: Tür ve Tarihe Göre Kiralama Sayısı ve Gelir

In [472]:
film_cat = film_category.merge(category, on="category_id").merge(film, on="film_id")
film_rentals = rental.merge(inventory, on="inventory_id").merge(payment, on="rental_id").merge(film_cat, on="film_id")

film_rentals['rental_day'] = pd.to_datetime(film_rentals['rental_date']).dt.date

result = film_rentals.groupby(['name','rental_day']).agg(
    rental_count=('rental_id','count'),
    total_revenue=('amount','sum')
).reset_index()
result.head()


,name,rental_day,rental_count,total_revenue
0,Action,2005-05-25,10,46.90
1,Action,2005-05-26,15,80.85
2,Action,2005-05-27,9,37.91
3,Action,2005-05-28,20,68.80
4,Action,2005-05-29,12,42.88


## Soru 12: Kiralanabilir Filmler İçin Türlerine Göre Her Filmin Kaç Kez Kiralandığı

In [475]:
film_cat = film_category.merge(category, on="category_id").merge(film, on="film_id")
film_rentals = rental.merge(inventory, on="inventory_id").merge(film_cat, on="film_id")

result = film_rentals.groupby(['name','title']).agg(
    rental_count=('rental_id','count')
).reset_index()
result.head()


,name,title,rental_count
0,Action,AMADEUS HOLY,21
1,Action,AMERICAN CIRCUS,22
2,Action,ANTITRUST TOMATOES,10
3,Action,BAREFOOT MANCHURIAN,18
4,Action,BERETS AGENT,21


## Soru 13: En çok rafta bekleyen filmler

In [478]:
inv_rental = inventory.merge(film_df, on="film_id", how="left").merge(rental, on="inventory_id", how="left")
waiting = inv_rental[inv_rental['rental_id'].isna()]

result = waiting.groupby('title')['inventory_id'].count().reset_index(name='waiting_count') \
                .sort_values('waiting_count', ascending=False)
result.head()


,title,waiting_count
0,ACADEMY DINOSAUR,1


## Soru 14: Geç, Erken ve Zamanında İade Edilen Kiralanmış Filmler

In [481]:
rental_with_film = rental_df.merge(inventory_df, on="inventory_id") \
                            .merge(film_df[["film_id","rental_duration"]], on="film_id")

rental_with_film["due_date"] = pd.to_datetime(rental_with_film["rental_date"]) + \
                               pd.to_timedelta(rental_with_film["rental_duration"], unit="D")
rental_with_film["return_date"] = pd.to_datetime(rental_with_film["return_date"])

def status(row):
    if pd.isna(row["return_date"]):
        return "NotReturned"
    elif row["return_date"] > row["due_date"]:
        return "Late"
    elif row["return_date"] < row["due_date"]:
        return "Early"
    else:
        return "OnTime"

rental_with_film["return_status"] = rental_with_film.apply(status, axis=1)
q14 = rental_with_film[["rental_id","return_status"]]
print(q14)

       rental_id return_status
0              1         Early
1              2         Early
2              3          Late
3              4          Late
4              5          Late
...          ...           ...
16039      16045         Early
16040      16046         Early
16041      16047         Early
16042      16048          Late
16043      16049          Late

[16044 rows x 2 columns]


## Soru 15: Hangi müşteri en çok DVD kiralamış?

In [484]:
rental = pd.read_sql_query("SELECT * FROM rental", conn)

q15 = (rental.groupby("customer_id")["rental_id"]
              .count()
              .reset_index(name="rental_count")
              .sort_values("rental_count", ascending=False)
              .head(1))
print(q15)


     customer_id  rental_count
147          148            46


## Soru 16: En popüler film kategorisi nedir?

In [487]:
film_cat = film_category.merge(category, on='category_id').merge(film_df, on='film_id')
film_rentals = rental.merge(inventory, on='inventory_id').merge(film_cat, on='film_id')

category_counts = film_rentals.groupby('name')['rental_id'].count().reset_index(name='rental_count')
category_counts.sort_values('rental_count', ascending=False).head(1)


,name,rental_count
14,Sports,1179


## Soru 17: Hangi çalışan en çok kiralama işlemi gerçekleştirmiş?

In [490]:
staff_counts = rental.groupby('staff_id')['rental_id'].count().reset_index(name='rental_count')
staff_counts.merge(staff, on='staff_id').sort_values('rental_count', ascending=False).head(1)


,staff_id,rental_count,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,8040,Mike,Hillyer,3,None,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2021-03-06 15:52:00


## Soru 18: En çok geliri hangi film getirmiş?

In [493]:
payment = pd.read_sql_query("SELECT * FROM payment", conn)
rental = pd.read_sql_query("SELECT * FROM rental", conn)
inventory = pd.read_sql_query("SELECT * FROM inventory", conn)
film = pd.read_sql_query("SELECT * FROM film", conn)


film = film[["film_id", "title"]]
inventory = inventory[["inventory_id", "film_id"]]

df18 = payment.merge(rental, on="rental_id") \
              .merge(inventory, on="inventory_id") \
              .merge(film, on="film_id")

q18 = (df18.groupby("title")["amount"]
              .sum()
              .reset_index(name="total_revenue")
              .sort_values("total_revenue", ascending=False)
              .head(1))

print(q18)


                title  total_revenue
841  TELEGRAPH VOYAGE         231.73


## Soru 19: Her müşteri için toplam harcama miktarını bulun

In [496]:
q19 = (payment.groupby("customer_id")["amount"]
               .sum()
               .reset_index(name="total_spent"))
print(q19.head())


   customer_id  total_spent
0            1       118.68
1            2       128.73
2            3       135.74
3            4        81.78
4            5       144.62


## Soru 20: Her kategorideki toplam kiralama sayısını ve gelirleri bulun

In [499]:
rental = rental[["rental_id", "inventory_id"]]
inventory = inventory[["inventory_id", "film_id"]]
film = film[["film_id"]]  # sadece film_id lazım
film_category = film_category[["film_id", "category_id"]]
category = category[["category_id", "name"]]
payment = payment[["rental_id", "amount"]]

df20 = rental.merge(inventory, on="inventory_id") \
             .merge(film, on="film_id") \
             .merge(film_category, on="film_id") \
             .merge(category, on="category_id") \
             .merge(payment, on="rental_id")

q20 = (df20.groupby("name")
              .agg(rental_count=("rental_id", "count"),
                   total_revenue=("amount", "sum"))
              .reset_index())

print(q20.head())

        name  rental_count  total_revenue
0     Action          1112        4375.85
1  Animation          1166        4656.30
2   Children           945        3655.55
3   Classics           939        3639.59
4     Comedy           941        4383.58


## Soru 21 En uzun süre kirada kalmış filmleri bulun

In [502]:
rental = pd.read_sql_query("SELECT * FROM rental", conn)
film = pd.read_sql_query("SELECT film_id, title FROM film", conn)
inventory = pd.read_sql_query("SELECT inventory_id, film_id FROM inventory", conn)

# rental süresi hesapla
rental["rental_duration"] = pd.to_datetime(rental["return_date"]) - pd.to_datetime(rental["rental_date"])

df21 = rental.merge(inventory, on="inventory_id") \
             .merge(film, on="film_id")

q21 = df21.sort_values("rental_duration", ascending=False).head(5)[["title", "rental_duration"]]
print(q21.head())

                    title rental_duration
14673     HIGHBALL POTTER 9 days 05:59:00
2409   HOLOCAUST HIGHBALL 9 days 05:59:00
7870           PANIC CLUB 9 days 05:58:00
13942          MASK PEACH 9 days 05:58:00
11624   CONEHEADS SMOOCHY 9 days 05:58:00


## Soru 22 En az kiralanan 5 film hangileridir?

In [505]:
df22 = rental.merge(inventory, on="inventory_id") \
             .merge(film, on="film_id")

q22 = (df22.groupby("title")["rental_id"]
              .count()
              .reset_index(name="rental_count")
              .sort_values("rental_count", ascending=True)
              .head(5))

In [507]:
print(q22.head())

                  title  rental_count
558         MIXED DOORS             4
866         TRAIN BUNCH             4
378      HARDLY ROBBERS             4
585  MUSSOLINI SPOILERS             5
315   FREEDOM CLEOPATRA             5


## Soru 24 En fazla kazanç sağlayan 5 müşteriyi bulun

In [510]:
payment = payment[["rental_id", "amount"]]
rental = rental[["rental_id", "customer_id"]]

df24 = payment.merge(rental, on="rental_id")

q24 = (df24.groupby("customer_id")["amount"]
              .sum()
              .reset_index(name="total_spent")
              .sort_values("total_spent", ascending=False)
              .head(5))

print(q24)


     customer_id  total_spent
525          526       221.55
147          148       216.54
143          144       195.58
136          137       194.61
177          178       194.61


## Soru 25 Her filmin ortalama kiralanma süresini bulun

In [513]:
rental_with_film = rental_df.merge(inventory_df, on="inventory_id") \
                            .merge(film_df[["film_id","title"]], on="film_id")

rental_with_film = rental_with_film[rental_with_film["return_date"].notna()]
rental_with_film["duration"] = (
    pd.to_datetime(rental_with_film["return_date"]) - pd.to_datetime(rental_with_film["rental_date"])
).dt.days

q25 = rental_with_film.groupby("title")["duration"].mean().reset_index(name="avg_duration")
print(q25)

                 title  avg_duration
0     ACADEMY DINOSAUR      4.545455
1       ACE GOLDFINGER      5.333333
2     ADAPTATION HOLES      2.833333
3     AFFAIR PREJUDICE      4.363636
4          AFRICAN EGG      6.727273
..                 ...           ...
953     YOUNG LANGUAGE      4.000000
954         YOUTH KICK      4.666667
955       ZHIVAGO CORE      5.250000
956  ZOOLANDER FICTION      5.176471
957          ZORRO ARK      4.096774

[958 rows x 2 columns]


## Soru 26: Her türde en popüler filmi bulun

In [525]:
film_df_clean = film_df.drop(columns=["last_update"])
inventory_df_clean = inventory_df.drop(columns=["last_update"])

film_cat_join = (
    category_df.merge(film_category_df, on="category_id")
               .merge(film_df_clean, on="film_id")
               .merge(inventory_df_clean, on="film_id")
               .merge(rental_df, on="inventory_id")
               .merge(payment_df[["rental_id", "amount"]], on="rental_id")
)

film_cat_rentals = film_cat_join.groupby(["name","title"])["rental_id"].count().reset_index(name="rental_count")

q26 = film_cat_rentals.loc[film_cat_rentals.groupby("name")["rental_count"].idxmax()]
q26


,name,title,rental_count
46,Action,RUGRATS SHAKESPEARE,30
93,Animation,JUGGLER HARDLY,32
165,Children,ROBBERS JOON,31
231,Classics,TIMBERLAND SKY,31
292,Comedy,ZORRO ARK,31
353,Documentary,WIFE TURN,31
380,Drama,HOBBIT ALIEN,31
418,Family,APACHE DIVINE,31
536,Foreign,ROCKETEER MOTHER,33
568,Games,FORWARD TEMPLE,32


## Soru 27: Her türde en fazla gelir sağlayan filmi bulun

In [532]:
film_cat_revenue = film_cat_join.groupby(["name","title"])["amount"].sum().reset_index(name="total_revenue")

q27 = film_cat_revenue.loc[film_cat_revenue.groupby("name")["total_revenue"].idxmax()]
q27


,name,title,total_revenue
24,Action,FOOL MOCKINGBIRD,175.77
74,Animation,DOGMA FAMILY,178.70
125,Children,BACKLASH UNDEFEATED,158.81
229,Classics,STEEL SANTA,141.77
292,Comedy,ZORRO ARK,214.69
353,Documentary,WIFE TURN,223.69
408,Drama,TORQUE BOUND,198.72
467,Family,RANGE MOONWALKER,179.73
514,Foreign,INNOCENT USUAL,191.74
584,Games,MASSACRE USUAL,179.70


## Soru 28: En çok DVD iade etmeyen müşteriyi bulun

In [543]:
not_returned = rental_df[rental_df["return_date"].isna()]
q28 = not_returned.groupby("customer_id")["rental_id"].count().reset_index(name="not_returned")
q28 = q28.merge(customer_df, on="customer_id").sort_values("not_returned", ascending=False).head(1)
q28

,customer_id,not_returned,store_id,first_name,last_name,email,address_id,active,create_date,last_update
23,75,3,2,TAMMY,SANDERS,TAMMY.SANDERS@sakilacustomer.org,79,1,2006-02-14 22:04:36.000,2021-03-06 15:53:36


## Soru 29: En fazla kiralama yapan 5 çalışanı bulun

In [549]:
q29 = rental_df.groupby("staff_id")["rental_id"].count().reset_index(name="rental_count")
q29 = q29.merge(staff, on="staff_id").sort_values("rental_count", ascending=False).head(5)
q29

,staff_id,rental_count,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,8040,Mike,Hillyer,3,None,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2021-03-06 15:52:00
1,2,8004,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,8cb2237d0679ca88db6464eac60da96345513964,2021-03-06 15:52:00


## Soru 30: En fazla kiralama yapan 5 müşteri hangi şubeden kiralama yapmış?

In [552]:
# store_id customer'da yoksa inventory üzerinden bağla
cust_counts = rental_df.merge(customer_df, on="customer_id") \
                       .merge(inventory_df[['inventory_id','film_id']], on="inventory_id")

# burada store_id eksikse, sadece customer_id bazında en çok kiralayanları bulabiliriz
result = cust_counts.groupby('customer_id')['rental_id'].count().reset_index(name='rental_count')
result = result.sort_values('rental_count', ascending=False).head(5)

final_result = result.merge(customer_df, on='customer_id')
final_result


,customer_id,rental_count,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,148,46,1,ELEANOR,HUNT,ELEANOR.HUNT@sakilacustomer.org,152,1,2006-02-14 22:04:36.000,2021-03-06 15:53:37
1,526,45,2,KARL,SEAL,KARL.SEAL@sakilacustomer.org,532,1,2006-02-14 22:04:37.000,2021-03-06 15:53:40
2,144,42,1,CLARA,SHAW,CLARA.SHAW@sakilacustomer.org,148,1,2006-02-14 22:04:36.000,2021-03-06 15:53:37
3,236,42,1,MARCIA,DEAN,MARCIA.DEAN@sakilacustomer.org,240,1,2006-02-14 22:04:36.000,2021-03-06 15:53:38
4,75,41,2,TAMMY,SANDERS,TAMMY.SANDERS@sakilacustomer.org,79,1,2006-02-14 22:04:36.000,2021-03-06 15:53:36


## Soru 31: Her türde en az kiralanan filmi bulun

In [555]:
film_cat = film_category_df.merge(category_df, on="category_id").merge(film_df, on="film_id")
film_rentals = rental_df.merge(inventory_df, on="inventory_id").merge(film_cat, on="film_id")

counts = film_rentals.groupby(['name','title'])['rental_id'].count().reset_index(name='rental_count')
result = counts.sort_values(['name','rental_count'], ascending=[True, True]).groupby('name').head(1)
result


,name,title,rental_count
27,Action,GOSFORD DONNIE,8
78,Animation,DUCK RACER,6
141,Children,FULL FLATLINERS,5
189,Classics,CONSPIRACY SPIRIT,5
254,Comedy,FREEDOM CLEOPATRA,5
315,Documentary,HARDLY ROBBERS,4
360,Drama,BUNCH MINDS,5
427,Family,BRAVEHEART HUMAN,5
525,Foreign,MIXED DOORS,4
566,Games,FEVER EMPIRE,5


## Soru 32: En çok kiralama yapan 5 müşteri hangi şehirde?

In [693]:
import pandas as pd 
customer = pd.read_sql_query("SELECT customer_id, address_id FROM customer", conn)
address = pd.read_sql_query("SELECT address_id, city_id FROM address", conn)
city = pd.read_sql_query("SELECT city_id, city FROM city", conn)

df32 = rental.merge(customer, on="customer_id") \
             .merge(address, on="address_id") \
             .merge(city, on="city_id")

top_customers = (df32.groupby("customer_id")["rental_id"]
                       .count()
                       .reset_index(name="rental_count")
                       .sort_values("rental_count", ascending=False)
                       .head(5))

q32 = top_customers.merge(df32[["customer_id", "city"]].drop_duplicates(),
                          on="customer_id")
print(q32.head(5))

   customer_id  rental_count         city
0          148            46  Saint-Denis
1          526            45   Cape Coral
2          144            42    Molodetno
3          236            42        Tanza
4           75            41     Changhwa


## Soru 33: En çok kazanç sağlayan 5 müşteri hangi şehirde bulunur?

In [563]:
payment = pd.read_sql_query("SELECT * FROM payment", conn)
customer = pd.read_sql_query("SELECT * FROM customer", conn)
address = pd.read_sql_query("SELECT * FROM address", conn)
city = pd.read_sql_query("SELECT * FROM city", conn)

In [695]:
df33 = payment.merge(customer[["customer_id", "address_id"]], on="customer_id") \
              .merge(address[["address_id", "city_id"]], on="address_id") \
              .merge(city[["city_id", "city"]], on="city_id")

customer_revenue = (df33.groupby("customer_id")["amount"]
                           .sum()
                           .reset_index(name="total_spent"))

top5 = customer_revenue.sort_values("total_spent", ascending=False).head(5)

q33 = top5.merge(df33[["customer_id", "city"]].drop_duplicates(),
                 on="customer_id")

print(q33.head(5))


   customer_id  total_spent                 city
0          526       221.55           Cape Coral
1          148       216.54          Saint-Denis
2          144       195.58            Molodetno
3          137       194.61            Apeldoorn
4          178       194.61  Santa Brbara dOeste


## Soru 34: En çok kiralanan 5 filmi hangi şehirde bulun?

In [697]:
rental = pd.read_sql_query("SELECT * FROM rental", conn)
inventory = pd.read_sql_query("SELECT * FROM inventory", conn)
film = pd.read_sql_query("SELECT film_id, title FROM film", conn)
customer = pd.read_sql_query("SELECT customer_id, address_id FROM customer", conn)
address = pd.read_sql_query("SELECT address_id, city_id FROM address", conn)
city = pd.read_sql_query("SELECT city_id, city FROM city", conn)

In [699]:
df34 = rental.merge(inventory[["inventory_id", "film_id"]], on="inventory_id") \
             .merge(film[["film_id", "title"]], on="film_id") \
             .merge(customer[["customer_id", "address_id"]], on="customer_id") \
             .merge(address[["address_id", "city_id"]], on="address_id") \
             .merge(city[["city_id", "city"]], on="city_id")

film_counts = (df34.groupby(["title", "city"])["rental_id"]
                      .count()
                      .reset_index(name="rental_count"))

top5_films = (film_counts.groupby("title")["rental_count"]
                        .sum()
                        .reset_index()
                        .sort_values("rental_count", ascending=False)
                        .head(5))

q34 = film_counts.merge(top5_films[["title"]], on="title")
print(q34.head(5))


                title             city  rental_count
0  BUCKET BROTHERHOOD          Allende             1
1  BUCKET BROTHERHOOD  Almirante Brown             1
2  BUCKET BROTHERHOOD         Alvorada             1
3  BUCKET BROTHERHOOD              Bag             1
4  BUCKET BROTHERHOOD            Batna             1


## Soru 35: En az kiralanan 5 filmi hangi şehirde bulun?

In [702]:
film_counts_all = (df34.groupby(["title", "city"])["rental_id"]
                           .count()
                           .reset_index(name="rental_count"))

least5_films = (film_counts_all.groupby("title")["rental_count"]
                             .sum()
                             .reset_index()
                             .sort_values("rental_count", ascending=True)
                             .head(5))

q35 = film_counts_all.merge(least5_films[["title"]], on="title")
print(q35.head(5))

               title                     city  rental_count
0  FREEDOM CLEOPATRA  Berhampore (Baharampur)             1
1  FREEDOM CLEOPATRA                Molodetno             1
2  FREEDOM CLEOPATRA                   Nagaon             1
3  FREEDOM CLEOPATRA                     York             1
4  FREEDOM CLEOPATRA                   al-Ayn             1


## Soru 36 En çok kazanç sağlayan 5 filmi hangi şehirde bulun

In [705]:
df36 = (payment_df[["payment_id","rental_id","amount","customer_id"]]
        .merge(rental_df[["rental_id","inventory_id","customer_id"]], on="rental_id")
        .merge(inventory_df[["inventory_id","film_id"]], on="inventory_id")
        .merge(film_df[["film_id","title"]], on="film_id")
        .merge(customer_df[["customer_id","address_id"]], on="customer_id")
        .merge(address_df[["address_id","city_id"]], on="address_id")
        .merge(city_df[["city_id","city"]], on="city_id"))

film_revenue = df36.groupby(["title","city"])["amount"].sum().reset_index(name="total_revenue")
top5_films = (film_revenue.groupby("title")["total_revenue"].sum()
              .reset_index().sort_values("total_revenue",ascending=False).head(5))
q36 = film_revenue.merge(top5_films[["title"]], on="title")
print(q36.head(5))

KeyError: 'customer_id'

## Soru 37 En az kazanç sağlayan 5 filmi hangi şehirde bulun

## Soru 38 En fazla kiralama yapan müşteri hangi filmleri kiralamış

In [707]:
top_customer = (rental.groupby("customer_id")["rental_id"]
                        .count()
                        .reset_index(name="rental_count")
                        .sort_values("rental_count", ascending=False)
                        .head(1))

df38 = rental.merge(inventory, on="inventory_id") \
             .merge(film[["film_id", "title"]], on="film_id")

q38 = df38[df38["customer_id"] == top_customer.iloc[0]["customer_id"]][["title"]].drop_duplicates()
q38

,title
680,PREJUDICE OLEANDER
1499,GUN BONNIE
1515,SNATCHERS MONTEZUMA
2748,ENGLISH BULWORTH
2840,FORWARD TEMPLE
2844,EGYPT TENENBAUMS
3650,MAJESTIC FLOATS
4077,FAMILY SWEET
4743,ARIZONA BANG
4947,REMEMBER DIARY


## Soru 40 En çok kazanç sağlayan müşteri hangi filmleri kiralamış

In [630]:
cust_revenue = payment_df.groupby("customer_id")["amount"].sum().reset_index(name="total_revenue")
top_customer_rev = cust_revenue.sort_values("total_revenue", ascending=False).iloc[0]["customer_id"]

q40 = (payment_df[payment_df["customer_id"]==top_customer_rev]
       .merge(rental_df[["rental_id","inventory_id"]], on="rental_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="inventory_id")
       .merge(film_df[["film_id","title"]], on="film_id"))
q40

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update,inventory_id,film_id,title
0,14136,526,1,495.0,4.99,2005-05-28 00:40:48.000,2021-03-06 15:57:53,1007,226,DESTINY SATURDAY
1,14137,526,2,679.0,4.99,2005-05-28 23:24:57.000,2021-03-06 15:57:53,907,201,CYCLONE FAMILY
2,14138,526,2,1015.0,2.99,2005-05-31 02:44:57.000,2021-03-06 15:57:53,3702,810,SLUMS DUCK
3,14139,526,1,1255.0,4.99,2005-06-15 06:13:45.000,2021-03-06 15:57:53,1433,313,FIDELITY DEVIL
4,14140,526,2,1848.0,0.99,2005-06-17 00:07:07.000,2021-03-06 15:57:53,3797,832,SPLASH GUMP
5,14141,526,2,1865.0,7.99,2005-06-17 01:49:36.000,2021-03-06 15:57:53,2657,583,MISSION ZOOLANDER
6,14142,526,2,1972.0,2.99,2005-06-17 09:25:49.000,2021-03-06 15:57:53,2758,605,MULHOLLAND BEAST
7,14143,526,1,1981.0,2.99,2005-06-17 10:03:34.000,2021-03-06 15:57:53,3186,698,PRINCESS GIANT
8,14144,526,2,2398.0,4.99,2005-06-18 15:56:53.000,2021-03-06 15:57:53,3001,658,PARIS WEEKEND
9,14145,526,1,2828.0,2.99,2005-06-19 20:51:33.000,2021-03-06 15:57:53,3230,709,RACER EGG


## Soru 41: En az kazanç sağlayan müşteri hangi filmleri kiralamış?

In [633]:
bottom_customer_rev = cust_revenue.sort_values("total_revenue", ascending=True).iloc[0]["customer_id"]

q41 = (payment_df[payment_df["customer_id"]==bottom_customer_rev]
       .merge(rental_df[["rental_id","inventory_id"]], on="rental_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="inventory_id")
       .merge(film_df[["film_id","title"]], on="film_id"))
q41

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update,inventory_id,film_id,title
0,6699,248,2,330.0,7.99,2005-05-27 02:15:30.000,2021-03-06 15:56:53,1536,336,FRENCH HOLIDAY
1,6700,248,1,618.0,4.99,2005-05-28 15:50:07.000,2021-03-06 15:56:53,4445,972,WHISPERER GIANT
2,6701,248,1,2066.0,3.99,2005-06-17 16:07:08.000,2021-03-06 15:56:53,2561,561,MASK PEACH
3,6702,248,2,2371.0,0.99,2005-06-18 14:35:29.000,2021-03-06 15:56:53,178,39,ARMAGEDDON LOST
4,6703,248,1,3910.0,0.99,2005-07-06 20:05:18.000,2021-03-06 15:56:53,2085,452,ILLUSION AMELIE
5,6704,248,2,4541.0,4.99,2005-07-08 04:04:19.000,2021-03-06 15:56:53,1976,431,HOOSIERS BIRDCAGE
6,6705,248,1,4841.0,0.99,2005-07-08 18:18:23.000,2021-03-06 15:56:53,518,114,CAMELOT VACATION
7,6706,248,1,5370.0,2.99,2005-07-09 18:43:19.000,2021-03-06 15:56:53,2193,474,JADE BUNCH
8,6707,248,2,6617.0,2.99,2005-07-12 08:39:56.000,2021-03-06 15:56:53,2292,498,KILLER INNOCENT
9,6708,248,2,7778.0,5.99,2005-07-28 07:10:11.000,2021-03-06 15:56:53,1044,233,DISCIPLE MOTHER


## Soru 42: En az kazanç sağlayan müşteri hangi türde en fazla film kiralamış?

In [637]:
q42 = (payment_df[payment_df["customer_id"]==bottom_customer_rev]
       .merge(rental_df[["rental_id","inventory_id"]], on="rental_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="inventory_id")
       .merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .groupby("name")["film_id"].count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=False)
       .head(1))
q42

,name,film_count
6,Sci-Fi,4


## Soru 43: En çok kiralanan film hangi çalışan tarafından kiralanmış?

In [652]:
staff_df = pd.read_sql("SELECT * FROM staff", conn)


In [662]:
q43 = (film_df[["film_id","title"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id","staff_id"]], on="inventory_id")
       .merge(staff_df[["staff_id","first_name","last_name"]], on="staff_id")
       .groupby(["title","first_name","last_name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q43

,title,first_name,last_name,rental_count
1408,ROCKETEER MOTHER,Jon,Stephens,23


## Soru 44: En az kiralanan film hangi çalışan tarafından kiralanmış?

In [709]:
### SQL sorgusu ile sonuç farklı çıktı

In [711]:
q44 = (film_df[["film_id","title"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id","staff_id"]], on="inventory_id")
       .merge(staff_df[["staff_id","first_name","last_name"]], on="staff_id")
       .groupby(["title","first_name","last_name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q44

,title,first_name,last_name,rental_count
1063,MANNEQUIN WORST,Mike,Hillyer,1


## Soru 45: En çok kazanç sağlayan film hangi çalışan tarafından kiralanmış?

In [670]:
q45 = (film_df[["film_id","title"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id","staff_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .merge(staff_df[["staff_id","first_name","last_name"]], on="staff_id")
       .groupby(["title","first_name","last_name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q45

,title,first_name,last_name,total_revenue
1680,TELEGRAPH VOYAGE,Jon,Stephens,147.83


## Soru 46: En az kazanç sağlayan film hangi çalışan tarafından kiralanmış?

In [713]:
### SQL sorgusu ile sonuç farklı çıktı

In [675]:
q46 = (film_df[["film_id","title"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id","staff_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .merge(staff_df[["staff_id","first_name","last_name"]], on="staff_id")
       .groupby(["title","first_name","last_name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q46

,title,first_name,last_name,total_revenue
630,FREEDOM CLEOPATRA,Mike,Hillyer,0.99


## Soru 47: En çok film kiralayan müşteri ve kiraladığı film sayısı

In [678]:
q47 = (rental_df.groupby("customer_id")["rental_id"].count()
       .reset_index(name="rental_count")
       .merge(customer_df[["customer_id","first_name","last_name"]], on="customer_id")
       .sort_values("rental_count", ascending=False)
       .head(1))
q47

,customer_id,rental_count,first_name,last_name
147,148,46,ELEANOR,HUNT


## Soru 48: En çok kazanç sağlayan müşteri ve ödediği toplam tutar

In [681]:
q48 = (payment_df.groupby("customer_id")["amount"].sum()
       .reset_index(name="total_payment")
       .merge(customer_df[["customer_id","first_name","last_name"]], on="customer_id")
       .sort_values("total_payment", ascending=False)
       .head(1))
q48

,customer_id,total_payment,first_name,last_name
525,526,221.55,KARL,SEAL


## Soru 49: En çok film kiralanan kategori

In [684]:
q49 = (category_df.merge(film_category_df, on="category_id")
       .merge(film_df[["film_id"]], on="film_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("name")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q49

,name,rental_count
14,Sports,1179


## Soru 50: En çok kazanç sağlanan kategori

In [687]:
q50 = (category_df.merge(film_category_df, on="category_id")
       .merge(film_df[["film_id"]], on="film_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("name")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q50

,name,total_revenue
14,Sports,5314.21


## Soru 51: En az film kiralayan müşteri

In [716]:
q51 = (rental_df.groupby("customer_id")["rental_id"].count()
       .reset_index(name="rental_count")
       .merge(customer_df[["customer_id","first_name","last_name"]], on="customer_id")
       .sort_values("rental_count", ascending=True)
       .head(1))
q51

,customer_id,rental_count,first_name,last_name
317,318,12,BRIAN,WYMAN


## Soru 52: En az kazanç sağlayan müşteri

In [719]:
q52 = (payment_df.groupby("customer_id")["amount"].sum()
       .reset_index(name="total_payment")
       .merge(customer_df[["customer_id","first_name","last_name"]], on="customer_id")
       .sort_values("total_payment", ascending=True)
       .head(1))
q52

,customer_id,total_payment,first_name,last_name
247,248,50.85,CAROLINE,BOWMAN


## Soru 53: En uzun filme sahip kategori

In [724]:
### SQl sorgusu ile sonuç farklı çıktı 

In [726]:
q53 = (category_df.merge(film_category_df, on="category_id")
       .merge(film_df[["film_id","length"]], on="film_id")
       .groupby("name")["length"]
       .max()
       .reset_index(name="max_length")
       .sort_values("max_length", ascending=False)
       .head(1))
q53


,name,max_length
0,Action,185


## Soru 54: En kısa filme sahip kategori

In [729]:
q54 = (category_df.merge(film_category_df, on="category_id")
       .merge(film_df[["film_id","length"]], on="film_id")
       .groupby("name")["length"]
       .min()
       .reset_index(name="min_length")
       .sort_values("min_length", ascending=True)
       .head(1))
q54

,name,min_length
2,Children,46


## Soru 55: En uzun film

In [732]:
q55 = film_df[["title","length"]].sort_values("length", ascending=False).head(1)
q55

,title,length
348,GANGS PRIDE,185


## Soru 56: En kısa film

In [735]:
q56 = film_df[["title","length"]].sort_values("length", ascending=True).head(1)
q56

,title,length
468,IRON MOON,46


## Soru 57: En çok film bulunan kategori

In [738]:
q57 = (category_df.merge(film_category_df, on="category_id")
       .groupby("name")["film_id"]
       .count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=False)
       .head(1))
q57

,name,film_count
14,Sports,74


## Soru 58: En az film bulunan kategori

In [741]:
q58 = (category_df.merge(film_category_df, on="category_id")
       .groupby("name")["film_id"]
       .count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=True)
       .head(1))
q58

,name,film_count
11,Music,51


## Soru 59: En çok kiralanan film uzunluğu (length)

In [744]:
q59 = (film_df[["film_id","length"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("length")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q59

,length,rental_count
39,85,305


## Soru 60: En çok kazanç sağlanan film uzunluğu (length)

In [750]:
q60 = (film_df[["film_id","length"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("length")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q60

,length,total_revenue
66,112,1235.57


## Soru 61: En az kiralanan film uzunluğu (length)

In [753]:
q61 = (film_df[["film_id","length"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("length")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q61

,length,rental_count
50,96,20


## Soru 62: En az kazanç sağlanan film uzunluğu (length)

In [756]:
q62 = (film_df[["film_id","length"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("length")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q62

,length,total_revenue
50,96,66.8


## Soru 63: En çok kiralanan film süresi (rental_duration)

In [761]:
q63 = (film_df[["film_id","rental_duration"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("rental_duration")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q63

,rental_duration,rental_count
0,3,3412


## Soru 64: En çok kazanç sağlanan film süresi (rental_duration)

In [765]:
q64 = (film_df[["film_id","rental_duration"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("rental_duration")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q64

,rental_duration,total_revenue
0,3,17310.81


## Soru 65: En az kiralanan film süresi (rental_duration)

In [768]:
q65 = (film_df[["film_id","rental_duration"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("rental_duration")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q65

,rental_duration,rental_count
4,7,2824


## Soru 66: En az kazanç sağlanan film süresi (rental_duration)

In [773]:
q66 = (film_df[["film_id","rental_duration"]]
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("rental_duration")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q66

,rental_duration,total_revenue
4,7,9392.84


## Soru 67: En çok film bulunan dil

In [778]:
language_df = pd.read_sql("SELECT * FROM language", conn)


In [780]:
q67 = (language_df.merge(film_df[["film_id","language_id"]], on="language_id")
       .groupby("name")["film_id"]
       .count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=False)
       .head(1))
q67

,name,film_count
0,English,1000


## Soru 68: En az film bulunan dil 

In [787]:
q68 = (language_df.merge(film_df[["film_id","language_id"]], on="language_id")
       .groupby("name")["film_id"]
       .count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=True)
       .head(1))
q68

,name,film_count
0,English,1000


## Soru 69: En çok film kiralanan dil 

In [790]:
q69 = (language_df.merge(film_df[["film_id","language_id"]], on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("name")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q69

,name,rental_count
0,English,16044


## Soru 70: En çok kazanç sağlanan dil

In [793]:
q70 = (language_df.merge(film_df[["film_id","language_id"]], on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("name")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q70

,name,total_revenue
0,English,67406.56


## Soru 71: En az film kiralanan dil

In [796]:
q71 = (language_df.merge(film_df[["film_id","language_id"]], on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby("name")["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q71

,name,rental_count
0,English,16044


## Soru 72: En az kazanç sağlanan dil

In [799]:
q72 = (language_df.merge(film_df[["film_id","language_id"]], on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby("name")["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q72

,name,total_revenue
0,English,67406.56


## Soru 73: En uzun film hangi dilde

In [802]:
q73 = (film_df.merge(language_df, on="language_id")
       .sort_values("length", ascending=False)
       [["title","name","length"]]
       .head(1))
q73

,title,name,length
348,GANGS PRIDE,English,185


## Soru 74: En kısa film hangi dilde

In [807]:
## Sonuç farklı çıktı 

In [809]:
q74 = (film_df.merge(language_df, on="language_id")
       .sort_values("length", ascending=True)
       [["title","name","length"]]
       .head(1))
q74

,title,name,length
468,IRON MOON,English,46


## Soru 75: En uzun kiralanan film süresi hangi dilde

In [812]:
q75 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["name","rental_duration"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values(["rental_duration","rental_count"], ascending=[False,False])
       .head(1))
q75

,name,rental_duration,rental_count
4,English,7,2824


## Soru 76: En kısa kiralanan film süresi hangi dilde

In [815]:
q76 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["name","rental_duration"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values(["rental_duration","rental_count"], ascending=[True,True])
       .head(1))
q76

,name,rental_duration,rental_count
0,English,3,3412


## Soru 77: En çok kazanç sağlanan film süresi (rental_duration) hangi dilde?

In [ ]:
q77 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["name","rental_duration"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q77

## Soru 78: En az kazanç sağlanan film süresi (rental_duration) hangi dilde?

In [821]:
q78 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["name","rental_duration"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q78

,name,rental_duration,total_revenue
4,English,7,9392.84


## Soru 79: En çok kiralanan film hangi dilde

In [825]:
q79 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["title","name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q79

,title,name,rental_count
96,BUCKET BROTHERHOOD,English,34


## Soru 80: En çok kazanç sağlayan film hangi dilde

In [828]:
q80 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["title","name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q80

,title,name,total_revenue
841,TELEGRAPH VOYAGE,English,231.73


## Soru 81: En az kiralanan film hangi dilde

In [ ]:
## Sonuç farklı çıktı 

In [833]:
q81 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["title","name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q81

,title,name,rental_count
558,MIXED DOORS,English,4


## Soru 82: En az kazanç sağlayan film hangi dilde

In [839]:
## Sonuç farklı çıktı 

In [841]:
q82 = (film_df.merge(language_df, on="language_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["title","name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q82

,title,name,total_revenue
847,TEXAS WATCH,English,5.94


## Soru 83: En uzun film hangi kategoride

In [847]:
## Sonuç farklı çıktı 

In [849]:
q83 = (film_df.merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .sort_values("length", ascending=False)
       [["title","name","length"]]
       .head(1))
q83

,title,name,length
348,GANGS PRIDE,Animation,185


## Soru 84: En kısa film hangi kategoride

In [857]:
## Sonuç farklı çıktı 

In [859]:
q84 = (film_df.merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .sort_values("length", ascending=True)
       [["title","name","length"]]
       .head(1))
q84

,title,name,length
468,IRON MOON,Classics,46


## Soru 85: En çok kiralanan film hangi kategoride

In [862]:
q85 = (film_df.merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["title","name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q85

,title,name,rental_count
96,BUCKET BROTHERHOOD,Travel,34


## Soru 86: En çok kazanç sağlayan film hangi kategoride 

In [866]:
q86 = (film_df.merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["title","name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q86

,title,name,total_revenue
841,TELEGRAPH VOYAGE,Music,231.73


## Soru 87: En az kiralanan film hangi kategoride

In [871]:
## Sonuç farklı çıktı 

In [873]:
q87 = (film_df.merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["title","name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q87

,title,name,rental_count
558,MIXED DOORS,Foreign,4


## Soru 88: En az kazanç sağlayan film hangi kategoride

In [877]:
q88 = (film_df.merge(film_category_df, on="film_id")
       .merge(category_df, on="category_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["title","name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q88

,title,name,total_revenue
847,TEXAS WATCH,Horror,5.94


## Soru 89: En uzun film hangi aktörle oynanmış?

In [882]:
## Sonuç farklı çıktı 

In [884]:
q89 = (film_df.merge(film_actor_df, on="film_id")
       .merge(actor_df, on="actor_id")
       .sort_values("length", ascending=False)
       [["title","first_name","last_name","length"]]
       .head(1))
q89

,title,first_name,last_name,length
5415,WORST BANGER,KIRSTEN,AKROYD,185


## Soru 90: En kısa film hangi aktörle oynanmış

In [887]:
q90 = (film_df.merge(film_actor_df, on="film_id")
       .merge(actor_df, on="actor_id")
       .sort_values("length", ascending=True)
       [["title","first_name","last_name","length"]]
       .head(1))
q90

,title,first_name,last_name,length
2739,KWAI HOMEWARD,FAY,WOOD,46


## Soru 91: En çok kiralanan film hangi aktörle oynanmış

In [890]:
q91 = (film_df.merge(film_actor_df, on="film_id")
       .merge(actor_df, on="actor_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["title","first_name","last_name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=False)
       .head(1))
q91

,title,first_name,last_name,rental_count
524,BUCKET BROTHERHOOD,BURT,TEMPLE,34


## Soru 92: En çok kazanç sağlayan film hangi aktörle oynanmış?

In [893]:
q92 = (film_df.merge(film_actor_df, on="film_id")
       .merge(actor_df, on="actor_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["title","first_name","last_name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q92

,title,first_name,last_name,total_revenue
4621,TELEGRAPH VOYAGE,VIVIEN,BASINGER,231.73


## Soru 93: En az kiralanan film hangi aktörle oynanmış?

In [901]:
## Sonuç farklı çıktı

In [903]:
q93 = (film_df.merge(film_actor_df, on="film_id")
       .merge(actor_df, on="actor_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .groupby(["title","first_name","last_name"])["rental_id"]
       .count()
       .reset_index(name="rental_count")
       .sort_values("rental_count", ascending=True)
       .head(1))
q93

,title,first_name,last_name,rental_count
4753,TRAIN BUNCH,RIP,CRAWFORD,4


## Soru 94: En az kazanç sağlayan film hangi aktörle oynanmış?

In [906]:
### Sonuç farklı çıktı

In [907]:
q94 = (film_df.merge(film_actor_df, on="film_id")
       .merge(actor_df, on="actor_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["title","first_name","last_name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=True)
       .head(1))
q94

,title,first_name,last_name,total_revenue
4654,TEXAS WATCH,SYLVESTER,DERN,5.94


## Soru 95: En çok filmde oynayan aktör

In [911]:
q95 = (film_actor_df.merge(actor_df, on="actor_id")
       .groupby(["first_name","last_name"])["film_id"]
       .count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=False)
       .head(1))
q95

,first_name,last_name,film_count
180,SUSAN,DAVIS,54


## Soru 96: En az filmde oynayan aktör

In [925]:
q96 = (film_actor_df.merge(actor_df, on="actor_id")
       .groupby(["first_name","last_name"])["film_id"]
       .count()
       .reset_index(name="film_count")
       .sort_values("film_count", ascending=True)
       .head(1))
q96

,first_name,last_name,film_count
51,EMILY,DEE,14


## Soru 97: En uzun filmde oynayan aktör



In [934]:
## Sonuç farklı çıktı 

In [936]:
q97 = (film_actor_df.merge(actor_df, on="actor_id")
       .merge(film_df[["film_id","title","length"]], on="film_id")
       .sort_values("length", ascending=False)
       .head(1))
q97

,actor_id,film_id,last_update_x,first_name,last_name,last_update_y,title,length
1965,74,872,2021-03-06 15:53:00,MILLA,KEITEL,2021-03-06 15:51:59,SWEET BROTHERHOOD,185


## Soru 98: En kısa filmde oynayan aktör

In [941]:
## Sonuç farklı çıktı 

In [943]:
q98 = (film_actor_df.merge(actor_df, on="actor_id")
       .merge(film_df[["film_id","title","length"]], on="film_id")
       .sort_values("length", ascending=True)
       .head(1))
q98

,actor_id,film_id,last_update_x,first_name,last_name,last_update_y,title,length
4617,169,505,2021-03-06 15:53:21,KENNETH,HOFFMAN,2021-03-06 15:52:00,LABYRINTH LEAGUE,46


## Soru 99: En çok kazanç sağlayan aktör

In [948]:
## Sonuç farklı çıktı 

In [950]:
q99 = (film_actor_df.merge(actor_df, on="actor_id")
       .merge(film_df[["film_id"]], on="film_id")
       .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
       .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
       .merge(payment_df[["rental_id","amount"]], on="rental_id")
       .groupby(["first_name","last_name"])["amount"]
       .sum()
       .reset_index(name="total_revenue")
       .sort_values("total_revenue", ascending=False)
       .head(1))
q99

,first_name,last_name,total_revenue
180,SUSAN,DAVIS,3538.74


## Soru 100: En az kazanç sağlayan aktör

In [954]:
q100 = (film_actor_df.merge(actor_df, on="actor_id")
        .merge(film_df[["film_id"]], on="film_id")
        .merge(inventory_df[["inventory_id","film_id"]], on="film_id")
        .merge(rental_df[["rental_id","inventory_id"]], on="inventory_id")
        .merge(payment_df[["rental_id","amount"]], on="rental_id")
        .groupby(["first_name","last_name"])["amount"]
        .sum()
        .reset_index(name="total_revenue")
        .sort_values("total_revenue", ascending=True)
        .head(1))
q100

,first_name,last_name,total_revenue
51,EMILY,DEE,883.85
